![picture_1.png](picture_1.png)


Considering the GPU architecture in the picture above, how many Stream 
Multiprocessors (SMs) does it have? And how many cores does it have? 

There is 16 SMs and 512 cores.

Considering that 1,2 and 3 are the representations of 3 different BLOCKS that someone is planning to execute simultaneously, which one(s) is(are) valid?

![picture_2.png](picture_2.png)

In [ ]:
import numpy as np 
import pycuda.autoinit 
import pycuda.driver as cuda 
from pycuda.compiler import SourceModule 
import time 

# Set GPU architecture for gtx 1060 
cuda.Context.get_device().make_context()
 
# Define the size of the vectors 
N = 360000000

# Create two random vectors and a destination vector 
a = np.random.randn(N).astype(np.float32) 
b = np.random.randn(N).astype(np.float32) 
c = np.zeros_like(a) 
 
# Allocate memory on the device and copy data from the host 
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(a_gpu, a) 
cuda.memcpy_htod(b_gpu, b) 
 
# Compile the kernel code 
mod = SourceModule(""" 
__global__ void vector_add_coarse_grained(float *a, float *b, float *c, int n) { 
    int index = (threadIdx.x + blockIdx.x * blockDim.x) * 2;  // Each thread will handle 2 elements 
    if (index < n) { 
        c[index] = a[index] + b[index]; 
        if (index + 1 < n) { 
            c[index + 1] = a[index + 1] + b[index + 1]; 
        } 
    } 
} 
 
__global__ void vector_add_fine_grained(float *a, float *b, float *c, int n) { 
    int index = threadIdx.x + blockIdx.x * blockDim.x; 
    if (index < n) { 
        c[index] = a[index] + b[index]; 
    } 
} 
""") 
 
# Get the kernel function from the compiled module 
vector_add_fine = mod.get_function("vector_add_fine_grained") 
vector_add_coarse = mod.get_function("vector_add_coarse_grained") 
 
# Launch the kernel 
threads_per_block = 1024 
blocks_per_grid = (N + threads_per_block - 1) // threads_per_block

# Choose one of these based on the granularity you want 
start_time = time.time() 
vector_add_fine(a_gpu, b_gpu, c_gpu, np.int32(N), block=(threads_per_block, 1, 1), grid=(blocks_per_grid, 1)) 
cuda.Context.synchronize()  # Wait for the kernel to finish 
fine_time = time.time() - start_time 
 
start_time = time.time() 
vector_add_coarse(a_gpu, b_gpu, c_gpu, np.int32(N), block=(threads_per_block, 1, 1), grid=(blocks_per_grid, 1)) 
cuda.Context.synchronize()  # Wait for the kernel to finish 
coarse_time = time.time() - start_time 
 
# Copy the result back to the host 
cuda.memcpy_dtoh(c, c_gpu) 
 
# Check the result 
if not np.allclose(c, a + b): 
    print("Discrepancy found in the results!") 
else: 
    print("Results are consistent with NumPy's computation.") 
 
    # Print the timing results 
print(f"Fine-grained kernel execution time: {fine_time:.6f} seconds") 
print(f"Coarse-grained kernel execution time: {coarse_time:.6f} seconds")

Results are consistent with NumPy's computation.\
Fine-grained kernel execution time: 0.017520 seconds\
Coarse-grained kernel execution time: 0.019040 seconds

In [2]:
mod = SourceModule("""
#include <stdio.h>
__global__ void vecAdd() {
    int i = blockIdx.x;
    printf("Hi from Block %d\\n", i);
}
""")
vecAdd = mod.get_function("vecAdd")
vecAdd(block=(12, 1, 1))
cuda.Context.synchronize()


Hi from Block 2\
Hi from Block 8\
Hi from Block 5\
Hi from Block 0\
Hi from Block 11\
Hi from Block 6\
Hi from Block 3\
Hi from Block 9\
Hi from Block 1\
Hi from Block 7\
Hi from Block 4\
Hi from Block 10


The Blocks were executed in a random order, this is due to the fact that the scheduler is responsible for the execution of the blocks and it is not possible to predict the order in which the blocks will be executed.

Write a CUDA kernel to perform element-wise addition of two vectors

In [ ]:
mod = SourceModule("""
__global__ void vector_add(float *a, float *b, float *c, int n) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    if (index < n) {
        c[index] = a[index] + b[index];
    }
}
""")

N = 1000000
a = np.random.randn(N).astype(np.float32)
b = np.random.randn(N).astype(np.float32)
c = np.zeros_like(a)

a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)

vector_add = mod.get_function("vector_add")
vector_add(a_gpu, b_gpu, c_gpu, np.int32(N), block=(1024, 1, 1), grid=(N // 1024 + 1, 1))
cuda.Context.synchronize()

cuda.memcpy_dtoh(c, c_gpu)

print(c)
print(a + b)


[ 1.5640146  -2.260321    0.89957166 ...  0.60166585 -0.6229739
  0.5706136 ]\
[ 1.5640146  -2.260321    0.89957166 ...  0.60166585 -0.6229739
  0.5706136 ]


Modify the vector addition kernel to use shared memory for one of the input vectors. 

In [ ]:
mod = SourceModule("""
__global__ void vector_add_shared(float *a, float *b, float *c, int n) {
    extern __shared__ float shared_a[];  // Declare shared memory for vector a
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    
    // Load data from global memory to shared memory
    if (threadIdx.x < n) {
        shared_a[threadIdx.x] = a[index];
    }
    __syncthreads();  // Synchronize threads to ensure all data is loaded
    
    // Perform element-wise addition using shared memory
    if (index < n) {
        c[index] = shared_a[threadIdx.x] + b[index];
    }
}
""")


block_size = 1024
grid_size = (N + block_size - 1) // block_size

vector_add_shared = mod.get_function("vector_add_shared")
vector_add_shared(a_gpu, b_gpu, c_gpu, np.int32(N), block=(block_size, 1, 1), grid=(grid_size, 1), shared=4 * block_size)
cuda.Context.synchronize()

cuda.memcpy_dtoh(c, c_gpu)

print(c)
print(a + b)

[ 1.5640146  -2.260321    0.89957166 ...  0.60166585 -0.6229739
  0.5706136 ]\
[ 1.5640146  -2.260321    0.89957166 ...  0.60166585 -0.6229739
  0.5706136 ]

Implement a tiled version of matrix multiplication using shared memory.

In [ ]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

# Define the matrix sizes
M = 1024
N = 1024
K = 1024

# Create random matrices A and B
A = np.random.randn(M, K).astype(np.float32)
B = np.random.randn(K, N).astype(np.float32)

# Allocate memory for the result matrix C
C = np.zeros((M, N), dtype=np.float32)

# Define the tile size
tile_size = 32

# Define the number of tiles
num_tiles = M // tile_size

# Define the shared memory size
shared_mem_size = tile_size * tile_size * 4

# Define the kernel code
kernel_code = """
__global__ void matrix_multiply_tiled(float* A, float* B, float* C, int M, int N, int K) {
    // Define shared memory for the tile
    __shared__ float tile_A[%(tile_size)s][%(tile_size)s];
    __shared__ float tile_B[%(tile_size)s][%(tile_size)s];

    // Calculate the global row and column indices
    int row = blockIdx.y * %(tile_size)s + threadIdx.y;
    int col = blockIdx.x * %(tile_size)s + threadIdx.x;

    // Initialize the result for this thread
    float result = 0.0;

    // Loop over the tiles of the input matrices
    for (int t = 0; t < %(num_tiles)s; t++) {
        // Load the tiles into shared memory
        tile_A[threadIdx.y][threadIdx.x] = A[row * K + t * %(tile_size)s + threadIdx.x];
        tile_B[threadIdx.y][threadIdx.x] = B[(t * %(tile_size)s + threadIdx.y) * N + col];

        // Synchronize to make sure the tiles are loaded
        __syncthreads();

        // Perform the matrix multiplication for the current tile
        for (int k = 0; k < %(tile_size)s; k++) {
            result += tile_A[threadIdx.y][k] * tile_B[k][threadIdx.x];
        }

        // Synchronize to make sure the result is updated
        __syncthreads();
    }

    // Write the result to the output matrix
    C[row * N + col] = result;
}
""" % {'tile_size': tile_size, 'num_tiles': num_tiles}

# Compile the kernel code
mod = pycuda.compiler.SourceModule(kernel_code)

# Get the kernel function from the compiled module
matrix_multiply_tiled = mod.get_function("matrix_multiply_tiled")

# Allocate memory on the GPU
A_gpu = cuda.mem_alloc(A.nbytes)
B_gpu = cuda.mem_alloc(B.nbytes)
C_gpu = cuda.mem_alloc(C.nbytes)

# Copy the input matrices to the GPU
cuda.memcpy_htod(A_gpu, A)
cuda.memcpy_htod(B_gpu, B)

# Define the grid and block dimensions
block_dim = (tile_size, tile_size, 1)
grid_dim = (N // tile_size, M // tile_size, 1)

# Launch the kernel
matrix_multiply_tiled(A_gpu, B_gpu, C_gpu, np.int32(M), np.int32(N), np.int32(K), block=block_dim, grid=grid_dim)

# Copy the result matrix back to the CPU
cuda.memcpy_dtoh(C, C_gpu)

# Print the result matrix
print(np.matmul(A, B))
print(C)


[[ 36.501495  -25.381676   32.539627  ... -11.418268   -9.357998\
  -17.958183 ]\
 [ 10.755127   -9.259977  -54.297485  ...  32.701073  -10.968725\
   -9.568348 ]\
 [ 39.094963   -7.4902244  33.729324  ...  36.56028    91.71763\
   19.633055 ]\
 ...\
 [-18.937649   -8.362484   18.537598  ...  18.997225   -3.654273\
  -23.108137 ]\
 [-28.292513  -45.573208   30.893597  ...  -5.900941   14.504358\
   32.278687 ]\
 [-34.52884    15.688816    8.461049  ...  34.496838  -29.627739\
  -23.486835 ]]\
[[ 36.501495  -25.381676   32.539635  ... -11.418284   -9.357988\
  -17.958187 ]\
 [ 10.755133   -9.259976  -54.297478  ...  32.701046  -10.9687195\
   -9.568327 ]\
 [ 39.09495    -7.4902477  33.7293    ...  36.56028    91.71768\
   19.633053 ]\
 ...\
 [-18.937649   -8.36248    18.537575  ...  18.997227   -3.6542697\
  -23.108149 ]\
 [-28.292515  -45.573162   30.893604  ...  -5.900946   14.504358\
   32.278698 ]\
 [-34.528816   15.688814    8.461073  ...  34.496845  -29.627726 \
  -23.486832 ]]


Perform an average on a 2D grid using shared memory to cache neighbouring elements.


Implement a parallel reduction in shared memory.

In [ ]:
import pycuda.autoinit
import numpy as np
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
# Define the input data
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=np.int32)

# Allocate memory on the GPU
data_gpu = cuda.mem_alloc(data.nbytes)

# Copy the input data to the GPU
cuda.memcpy_htod(data_gpu, data)

# Define the kernel code
kernel_code = """
__global__ void parallel_reduction(int* data, int* result) {
    // Define shared memory for the block
    __shared__ int shared_data[%(block_size)s];

    // Calculate the global index
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    // Load the data into shared memory
    shared_data[threadIdx.x] = data[index];

    // Synchronize to make sure the data is loaded
    __syncthreads();

    // Perform the reduction
    for (unsigned int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            shared_data[threadIdx.x] += shared_data[threadIdx.x + stride];
        }
        __syncthreads();
    }

    // Store the result in global memory
    if (threadIdx.x == 0) {
        result[blockIdx.x] = shared_data[0];
    }
}
""" % {'block_size': len(data)}

# Compile the kernel code
mod = SourceModule(kernel_code)

# Get the kernel function from the compiled module
parallel_reduction = mod.get_function("parallel_reduction")

# Define the block and grid dimensions
block_dim = (len(data), 1, 1)
grid_dim = (1, 1, 1)

# Allocate memory for the result
result = np.zeros(1, dtype=np.int32)
result_gpu = cuda.mem_alloc(result.nbytes)

# Launch the kernel
parallel_reduction(data_gpu, result_gpu, block=block_dim, grid=grid_dim)

# Copy the result back to the CPU
cuda.memcpy_dtoh(result, result_gpu)

# Print the result
print("Sum:", result[0])


Sum: 40
